!!!这个示例没有跑通，需要调整。从学习思路的角度看是极好的

In [ ]:
import './../../loadenv.mjs'

# Create Tools

## Research team tools

In [2]:
import { TavilySearchResults } from '@langchain/community/tools/tavily_search'
import { CheerioWebBaseLoader } from '@langchain/community/document_loaders/web/cheerio'
import { tool } from '@langchain/core/tools'
import { z } from 'zod'
import { zodToJsonSchema } from 'zod-to-json-schema'

const tavilyTool = new TavilySearchResults()
const scrapeWebpage = tool(
    async (input: { url: string;}) => {
        const loader = new CheerioWebBaseLoader(input.url)
        const docs = await loader.load()
        const formattedDocs = docs.map(
            doc => `<Document name="${doc.metadata?.title}">\n${doc.pageContent}\n</Document>`
        )
        return formattedDocs.join('\n\n')
    },
    {
        name: 'scrape_webpage',
        description: 'Scrape the contents of a webpage',
        schema: z.object({
            url: z.string()
        })
    }
)

In [ ]:
import { inspect } from "node:util";
console.log(inspect(zodToJsonSchema(
    z.object({
        file_name: z.string(),
        inserts: z.tuple([
            z.number().describe('line number to insert at'),
            z.string().describe('content to insert'),
        ]).array().describe('insert items'),
    })
), { showHidden: false, depth: null, colors: true }))

## Document writing team tools

In [4]:
import { chartTool } from './../../utils.mjs'
import * as fs from 'node:fs/promises'
import * as path from 'node:path'

const WORKING_DIRECTORY = './temp'
await fs.mkdir(WORKING_DIRECTORY, { recursive: true })

const createOutlineTool = tool(
    async ({ points, file_name }: { points: string[]; file_name: string; }) => {
        const filePath = path.join(WORKING_DIRECTORY, file_name)
        const data = points
            .map((point: string, index: number) => `${index + 1}. ${point}\n`)
            .join('')
        await fs.writeFile(filePath, data)
        return `Outline saved to ${file_name}`
    },
    {
        name: 'create_outline',
        description: 'Create and save an outline',
        schema: z.object({
            points: z
                .array(z.string())
                .nonempty('List of main points or sections must not be empty.'),
            file_name: z.string()
        })
    }
)
 
const readDocumentTool = tool(
    async ({ file_name, start, end }) => {
        const filePath = path.join(WORKING_DIRECTORY, file_name)
        const data = await fs.readFile(filePath, 'utf-8')
        const lines = data.split('\n')
        return lines.slice(start ?? 0, end).join('\n')
    },
    {
        name: 'read_document',
        description: 'Read the specified document.',
        schema: z.object({
            file_name: z.string(),
            start: z.number().optional(),
            end: z.number().optional(),
        })
    }
)

const writeDocumentTool = tool(
    async ({ content, file_name }: { content: string; file_name: string; }) => {
        const filePath = path.join(WORKING_DIRECTORY, file_name)
        await fs.writeFile(filePath, content)
        return `Document saved to ${file_name}`
    },
    {
        name: 'write_document',
        description: 'Create and save a text document',
        schema: z.object({
            content: z.string(),
            file_name: z.string(),
        }),
    }
)

type IEditDocumentToolParams = {
    file_name: string;
    insertLineNumbers: number[];
    insertContents: string[];
}
const editDocumentTool = tool(
    async ({ file_name, insertLineNumbers, insertContents }: IEditDocumentToolParams) => {
        const filePath = path.join(WORKING_DIRECTORY, file_name)
        const data = await fs.readFile(filePath, 'utf-8')
        const lines = data.split('\n')

        // const sortedInserts = Object.entries(inserts).sort(
        //     ([a], [b]) => parseInt(a) - parseInt(b)
        // )
        // const sortedInserts = inserts.sort(([a], [b]) => a - b)
        const sortedInserts: Array<[number, string]> = [];
        for (let i = 0; i < insertLineNumbers.length; i++) {
            sortedInserts.push([
                insertLineNumbers[i],
                insertContents[i],
            ])
        }
        for (const [line_number, text] of sortedInserts) {
            if (1 <= line_number && line_number <= lines.length + 1) {
                lines.splice(line_number - 1, 0, text)
            } else {
                return `Error: Line number ${line_number} is out of range.`
            }
        }

        await fs.writeFile(filePath, lines.join('\n'))
        return `Document edited and saved to ${file_name}`
    },
    {
        name: 'edit_document',
        description: 'Edit a document by inserting text at specific line numbers.',
        schema: z.object({
            file_name: z.string(),
            // 这里暂时不知道好不好
            insertLineNumbers: z.number().array().describe('line numbers to insert at, sorted'),
            insertContents: z.string().array().describe('contents to insert at, sorted by line_numbers'),
            // inserts: z.tuple([
            //     z.number().describe('line number to insert at'),
            //     z.string().describe('content to insert'),
            // ]).array().describe('insert items'),
        }),
    }
)

In [5]:
// await writeDocumentTool.invoke({
//     content: 'Hello from LangGraph!',
//     file_name: 'hello.txt',
// })

In [6]:
// await readDocumentTool.invoke({ file_name: 'hello.txt' })

In [7]:
// await chartTool.invoke({
//     data: [
//         { label: 'People who like graphs', value: 5000 },
//         { label: 'People who like LangGraph', value: 10000 },
//     ]
// })

# Helper Utilities

In [8]:
import { HumanMessage, BaseMessage, SystemMessage } from '@langchain/core/messages'
import { ChatPromptTemplate, MessagesPlaceholder } from '@langchain/core/prompts'
import { JsonOutputToolsParser } from 'langchain/output_parsers'
import { getModel } from './../../utils.mjs'
import { Runnable } from '@langchain/core/runnables'
import { StructuredToolInterface } from '@langchain/core/tools'
import { MessagesAnnotation } from '@langchain/langgraph'
import { ChatOpenAI } from '@langchain/openai'

const agentStateModifier = (
    systemPrompt: string,
    tools: StructuredToolInterface[],
    teamMembers: string[],
): ((state: typeof MessagesAnnotation.State) => BaseMessage[]) => {
    const toolNames = tools.map(t => t.name).join(', ')
    const systemMsgStart = new SystemMessage(systemPrompt +
        "\nWork autonomously according to your specialty, using the tools available to you." +
        " Do not ask for clarification." +
        " Your other team members (and other teams) will collaborate with you with their own specialties." +
        ` You are chosen for a reason! You are one of the following team members: ${teamMembers.join(", ")}.`
    )
    const systemMsgEnd = new SystemMessage(`Supervisor instructions: ${systemPrompt}\n` +
        `Remember, you individually can only use these tools: ${toolNames}` +
        "\n\nEnd if you have already completed the requested task. Communicate the work completed.")
    return (state: typeof MessagesAnnotation.State): any[] =>
    [systemMsgStart, ...state.messages, systemMsgEnd];
}

async function runAgentNode(params: {
    state: any;
    agent: Runnable;
    name: string;
}) {
    const { state, agent, name } = params
    const result = await agent.invoke({
        messages: state.messages,
    })
    const lastMessage = result.messages[result.messages.length - 1]
    return {
        messages: [new HumanMessage({ content: lastMessage.content, name })]
    }
}

async function createTeamSupervisor(
    llm: ChatOpenAI,
    systemPrompt: string,
    members: string[],
): Promise<Runnable> {
    const options = ['FINISH', ...members]
    const routeTool = {
        name: 'route',
        description: 'Select the next role.',
        schema: z.object({
            reasoning: z.string(),
            next: z.enum(['FINISH', ...members]),
            instructions: z.string().describe("The specific instructions of the sub-task the next role should accomplish.")
        })
    }
    let prompt = ChatPromptTemplate.fromMessages([
        ['system', systemPrompt],
        new MessagesPlaceholder('messages'),
        [
            'system',
            'Given the conversation above, who should act next? Or should we FINISH? Select one of: {options}',
        ]
    ])
    prompt = await prompt.partial({
        options: options.join(', '),
        team_members: members.join(', '),
    })

    const supervisor = prompt
        .pipe(
            llm.bindTools([routeTool], {
                tool_choice: 'route',
            })
        )
        .pipe(new JsonOutputToolsParser())
        .pipe(x => ({
            next: x[0].args.next,
            instructions: x[0].args.instructions,
        }))
    return supervisor
}

# Define Agent Teams

## Research Team

In [9]:
import { Annotation } from '@langchain/langgraph'
import { createReactAgent } from '@langchain/langgraph/prebuilt'

const ResearchTeamState = Annotation.Root({
    messages: Annotation<BaseMessage[]>({
        reducer: (x, y) => x.concat(y),
    }),
    team_members: Annotation<string[]>({
        reducer: (x, y) => x.concat(y),
    }),
    next: Annotation<string>({
        reducer: (x, y) => y ?? x,
        default: () => 'supervisor',
    }),
    instructions: Annotation<string>({
        reducer: (x, y) => y ?? x,
        default: () => 'Solve the human\'s question.',
    }),
})

const llm = getModel()

const searchNode = (state: typeof ResearchTeamState.State) => {
    const stateModifier = agentStateModifier(
        'You are a research assistant who can search for up-to-date info using the tavily search engine.',
        [tavilyTool],
        state.team_members ?? ['Search'],
    )
    const searchAgent = createReactAgent({
        llm,
        tools: [tavilyTool],
        stateModifier,
    })
    return runAgentNode({ state, agent: searchAgent, name: 'Search' })
}

const researchNode = (state: typeof ResearchTeamState.State) => {
    const stateModifier = agentStateModifier(
        'You are a research assistant who can scrape specified urls for more detailed information using the scrapeWebpage function.',
        [scrapeWebpage],
        state.team_members ?? ['WebScraper'],
    )
    const researchAgent = createReactAgent({
        llm,
        tools: [scrapeWebpage],
        stateModifier,
    })
    return runAgentNode({
        state,
        agent: researchAgent,
        name: 'WebScraper',
    })
}

const supervisorAgent = await createTeamSupervisor(
    llm,
    "You are a supervisor tasked with managing a conversation between the" +
        " following workers:  {team_members}. Given the following user request," +
        " respond with the worker to act next. Each worker will perform a" +
        " task and respond with their results and status. When finished," +
        " respond with FINISH.\n\n" +
        " Select strategically to minimize the number of steps taken.",
    ['Search', 'WebScraper'],
)

In [10]:
import { END, START, StateGraph } from '@langchain/langgraph'

const researchGraph = new StateGraph(ResearchTeamState)
    .addNode('Search', searchNode)
    .addNode('supervisor', supervisorAgent)
    .addNode('WebScraper', researchNode)
    .addEdge('Search', 'supervisor')
    .addEdge('WebScraper', 'supervisor')
    .addConditionalEdges('supervisor', (x) => x.next, {
        Search: 'Search',
        WebScraper: 'WebScraper',
        FINISH: END,
    })
    .addEdge(START, 'supervisor')

const researchChain = researchGraph.compile()

In [ ]:
import { printGraph } from './../../utils.mjs'
await printGraph(researchChain.getGraph())

In [12]:
// const streamResults = researchChain.stream(
//     {
//         messages: [new HumanMessage("What's the price of a big mac in Argentina?")],
//     },
//     {
//         recursionLimit: 100
//     }
// )
// for await (const output of await streamResults) {
//     if (!output?.__end__) {
//         console.log(output)
//         console.log('----')
//     }
// }

## Document Writing Team

In [13]:
import { RunnableLambda } from '@langchain/core/runnables'

const prelude = new RunnableLambda({
    func: async (state: {
        messages: BaseMessage[];
        next: string;
        instructions: string;
    }) => {
        let writtenFiles: string[] = [];
        if (
            !(await fs
                .stat(WORKING_DIRECTORY)
                .then(() => true)
                .catch(() => false)
            )
        ) {
            await fs.mkdir(WORKING_DIRECTORY, { recursive: true })
        }
        try {
            const files = await fs.readdir(WORKING_DIRECTORY)
            for (const file of files) {
                writtenFiles.push(file)
            }
        } catch (error) {
            console.error(error)
        }
        const filesList = writtenFiles.length > 0
            ? '\nBelow are files your team has written to the directory:\n' +
                writtenFiles.map(f => ` - ${f}`).join('\n')
            : 'No files written.'
        return {
            ...state,
            current_files: filesList
        }
    }
})

In [14]:
const DocWritingState = Annotation.Root({
    messages: Annotation<BaseMessage[]>({
        reducer: (x, y) => x.concat(y),
        default: () => [],
    }),
    team_members: Annotation<string[]>({
        reducer: (x, y) => x.concat(y),
        default: () => [],
    }),
    next: Annotation<string>({
        reducer: (x, y) => y ?? x,
        default: () => 'supervisor',
    }),
    current_files: Annotation<string>({
        reducer: (x, y) => (y ? `${x}\n${y}` : x),
        default: () => 'No files written',
    }),
    instructions: Annotation<string>({
        reducer: (x, y) => y ?? x,
        default: () => 'Solve the human\'s question.',
    })
})

In [15]:
const docWritingLlm = getModel()

const docWritingNode = (state: typeof DocWritingState.State) => {
    const tools = [readDocumentTool, writeDocumentTool, editDocumentTool]
    const stateModifier = agentStateModifier(
        `You are an expert writing a research document.\nBelow are files currently in your directory:\n${state.current_files}`,
        tools,
        state.team_members ?? [],
    ) 
    const docWriterAgent = createReactAgent({
        llm: docWritingLlm,
        tools,
        stateModifier,
    })
    const contextAwareDocWriterAgent = prelude.pipe(docWriterAgent)
    return runAgentNode({
        state,
        agent: contextAwareDocWriterAgent,
        name: 'DocWriter',
    })
}

const noteTakingNode = (state: typeof DocWritingState.State) => {
    const stateModifier = agentStateModifier(
        'You are an expert senior researcher tasked with writing a paper outline and' +
            ` taking notes to craft a perfect paper. ${state.current_files}`,
        [createOutlineTool, readDocumentTool],
        state.team_members ?? [],
    )
    const noteTakingAgent = createReactAgent({
        llm: docWritingLlm,
        tools: [createOutlineTool, readDocumentTool],
        stateModifier,
    })
    const contextAwareNoteTakingAgent = prelude.pipe(noteTakingAgent)
    return runAgentNode({
        state,
        agent: contextAwareNoteTakingAgent,
        name: 'NoteTaker',
    })
}

const chartGeneratingNode = async (state: typeof DocWritingState.State) => {
    const stateModifier = agentStateModifier(
        'You are a data viz expert tasked with generating charts for a research project.\n'
            + `${state.current_files}`,
        [readDocumentTool, chartTool],
        state.team_members ?? [],
    )
    const chartGeneratingAgent = createReactAgent({
        llm: docWritingLlm,
        tools: [readDocumentTool, chartTool],
        stateModifier,
    })
    const contextAwareChartGeneratingAgent = prelude.pipe(chartGeneratingAgent)
    return runAgentNode({
        state,
        agent: contextAwareChartGeneratingAgent,
        name: 'ChartGenerator',
    })
}

const docTeamMembers = ['DocWriter', 'NoteTaker', 'ChartGenerator']
const docWritingSupervisor = await createTeamSupervisor(
    docWritingLlm,
    "You are a supervisor tasked with managing a conversation between the" +
        " following workers:  {team_members}. Given the following user request," +
        " respond with the worker to act next. Each worker will perform a" +
        " task and respond with their results and status. When finished," +
        " respond with FINISH.\n\n" +
        " Select strategically to minimize the number of steps taken.",
    docTeamMembers,
)

In [16]:
const authoringApp = new StateGraph(DocWritingState)
    .addNode('DocWriter', docWritingNode)
    .addNode('NoteTaker', noteTakingNode)
    .addNode('ChartGenerator', chartGeneratingNode)
    .addNode('supervisor', docWritingSupervisor)
    .addEdge('DocWriter', 'supervisor')
    .addEdge('NoteTaker', 'supervisor')
    .addEdge('ChartGenerator', 'supervisor')
    .addConditionalEdges('supervisor', (x) => x.next, {
        DocWriter: 'DocWriter',
        NoteTaker: 'NoteTaker',
        ChartGenerator: 'ChartGenerator',
        FINISH: END,
    })
    .addEdge(START, 'supervisor')

const authoringGraph = authoringApp.compile()


In [ ]:
await printGraph(authoringGraph.getGraph())

In [18]:
const enterAuthoringChain = RunnableLambda.from(
    ({ messages }: { messages: BaseMessage[] }) => {
        return {
            messages,
            team_members: ['DocWriter', 'NoteTaker', 'ChartGenerator'],
        }
    }
)
const authoringChain = enterAuthoringChain.pipe(authoringGraph)

评：下面这个任务， `Doubao`的表现没有`DeepSeek`好，但无论是哪个，都称不上准确
原因还是字符数量的统计出问题。还是得写一个读文件数字符的tool才行。
写诗存到文件中，以及按照乱统计出的字符数量画bar，倒是能正常运行

In [19]:
// let resultStream = authoringChain.stream(
//     {
//         messages: [
//             new HumanMessage(
//                 'Write a limerick and make a bar chart of characters used.\nLet\'s think step by step.',
//             )
//         ]
//     },
//     {
//         recursionLimit: 100,
//     }
// )
// for await (const step of await resultStream) {
//     console.log(step)
//     console.log('---')
// }

# Add Layers

In [20]:
const State = Annotation.Root({
    messages: Annotation<BaseMessage[]>({
        reducer: (x, y) => x.concat(y),
    }),
    next: Annotation<string>({
        reducer: (x, y) => y ?? x,
        default: () => 'ResearchTeam',
    }),
    instructions: Annotation<string>({
        reducer: (x, y) => y ?? x,
        default: () => 'Resolve the user\'s request.',
    })
})

const supervisorNode = await createTeamSupervisor(
    llm,
    "You are a supervisor tasked with managing a conversation between the" +
        " following teams: {team_members}. Given the following user request," +
        " respond with the worker to act next. Each worker will perform a" +
        " task and respond with their results and status. When finished," +
        " respond with FINISH.\n\n" +
        " Select strategically to minimize the number of steps taken.",
    ['ResearchTeam', 'PaperWritingTeam'],
)

const getMessages = RunnableLambda.from((state: typeof State.State) => {
    return {
        messages: state.messages,
    }
})

const joinGraph = RunnableLambda.from((response: any) => {
    return {
        messages: [response.messages[response.messages.length - 1]],
    }
})

In [21]:
const superGraph = new StateGraph(State)
    .addNode('ResearchTeam', async (input) => {
        const getMessagesResult = await getMessages.invoke(input)
        const researchChainResult = await researchChain.invoke({
            messages: getMessagesResult.messages,
        })
        const joinGraphResult = await joinGraph.invoke({
            messages: researchChainResult.messages,
        })
    })
    .addNode('PaperWritingTeam', getMessages.pipe(authoringChain).pipe(joinGraph))
    .addNode('supervisor', supervisorNode)
    .addEdge('ResearchTeam', 'supervisor')
    .addEdge('PaperWritingTeam', 'supervisor')
    .addConditionalEdges('supervisor', (x) => x.next, {
        PaperWritingTeam: 'PaperWritingTeam',
        ResearchTeam: 'ResearchTeam',
        FINISH: END,
    })
    .addEdge(START, 'supervisor')
const compiledSuperGraph = superGraph.compile()

In [ ]:
await printGraph(compiledSuperGraph.getGraph())

In [ ]:
let resultStream = compiledSuperGraph.stream(
    {
        messages: [
            new HumanMessage(
                '寻找一个当前热点事件, write a poem about it, then plot a bar chart of the distribution of words therein',
            )
        ]
    }
)
for await (const step of await resultStream) {
    if (!step.__end__) {
        console.log(step)
        console.log('---')
    }
}